libraries

In [ ]:
import pandas as pd
from pyts.image import MarkovTransitionField
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold

from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf
import os

from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import StratifiedKFold
import matplotlib.patches as patches
from scipy import interp

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import label_binarize
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from pyts.image import GramianAngularField

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D,Conv1D, Concatenate,Dense, Dropout, Flatten
import pandas as pd
import tensorflow as tf

#cnn model

##functions

In [ ]:
#Reset index
def res_ind(df):
  df=df.reset_index()
  df=df.drop(['index'], axis=1)
  return df

In [ ]:
## binary classification model
def Model_1():
  model = Sequential()
  model.add(Conv2D(32, (2, 2), activation='relu', input_shape=(16, 16,1)) )
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (2, 2), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',
                metrics=['accuracy','Precision','Recall'])
  return model

In [ ]:
def Kfloding(kf,model,x,y,floder_path,model_file,subject,epochs,batch_size):
  callback_2 = ModelCheckpoint(floder_path +str(subject)+'__'+str(model_file)+'.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
  callback_3 = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
  callback_1 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

  cv = StratifiedKFold(n_splits=int(kf),shuffle=True,random_state=98)
  #fig1 = plt.figure(figsize=[12,12])
  #ax1 = fig1.add_subplot(111,aspect = 'equal')
  accuracy_v=[]
  y_true=[]
  y_predict = []
  tprs = []
  aucs = []
  mean_fpr = np.linspace(0,1,100)
  i = 1
  for train_index,test_index in cv.split(x,y):
      x_train,x_test=x[train_index],x[test_index]
      y_train,y_test=y[train_index],y[test_index]
      model.fit(x_train, y_train,batch_size=batch_size,
                          epochs=epochs,
                          validation_data=(x_test,y_test),
                          callbacks=[callback_2])
      model = load_model(floder_path +str(subject)+'__'+str(model_file)+'.h5')
      print('Model evaluation ',model.evaluate(x_test,y_test))
      loss,accuracy,precision,recall_1=model.evaluate(x_test,y_test)
      print('kf_fold',i," ", 'loss',loss,'Accurary',accuracy,'Precision',precision,'Recall',recall_1)
      i=i+1
      accuracy_v.append(accuracy)
  return accuracy_v,sum(accuracy_v)/10


In [ ]:
def mark_img(dat):
  mtf = MarkovTransitionField(image_size=16)
  MTF = mtf.fit_transform(dat.iloc[:,:-1])
  x = MTF.reshape(MTF.shape[0], 16, 16,1)
  y=dat['label']
  return x,y


In [ ]:
def gram_diff(dat):
  gram = GramianAngularField(image_size=16, method='difference')
  gram_t = gram.fit_transform(dat.iloc[:,:-1])
  x = gram_t.reshape(gram_t.shape[0], 16, 16,1)
  y=dat['label']
  return x,y

In [ ]:
def gram_summ(dat):
  gram = GramianAngularField(image_size=16, method='summation')
  gram_t = gram.fit_transform(dat.iloc[:,:-1])
  x = gram_t.reshape(gram_t.shape[0], 16, 16,1)
  y=dat['label']
  return x,y

In [ ]:
def kf_flod_three(kf,x,y,floder_path,model_file,subject,epochs,batch_size):
    model1 = Sequential()
    model1.add(Conv2D(32, (2, 2), activation='relu', input_shape=(16, 16,1)))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, (2, 2), activation='relu'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Flatten())
    model1.add(Dense(32, activation='relu'))
    model1.add(Dense(8,activation='relu'))
    model1.add(Dense(3,activation='softmax'))
    model1.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy','Precision','Recall'])

    callback_2 = ModelCheckpoint(floder_path +str(subject)+'__'+str(model_file)+'.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    callback_3 = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
    callback_1 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

    cv = StratifiedKFold(n_splits=int(kf),shuffle=True,random_state=98)
    #fig1 = plt.figure(figsize=[12,12])
    #ax1 = fig1.add_subplot(111,aspect = 'equal')
    accuracy_v=[]
    y_true=[]
    y_predict = []
    pred_probs=[]
    tprs = []
    aucs = []
    fold_no=0
    #y_tf = tf.keras.utils.to_categorical(y, num_classes = 3)
    i=1
    for train_index,test_index in cv.split(x,y):
        x_train,x_test=x[train_index],x[test_index]
        y_train,y_test=y[train_index],y[test_index]
        y_train = tf.keras.utils.to_categorical(y_train, num_classes = 3)
        y_test = tf.keras.utils.to_categorical(y_test, num_classes = 3)

        model1.fit(x_train, y_train,batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(x_test,y_test),
                            callbacks=[callback_2])
        model1 = load_model(floder_path +str(subject)+'__'+str(model_file)+'.h5')
        #scores = model.evaluate(x_test, y_test, verbose=0)
        #print(scores)
        #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        #   print("***************************************************\n")
        loss,accuracy,precision,recall_1=model1.evaluate(x_test,y_test)
        print('kf_fold',i," ", 'loss',loss,'Accurary',accuracy,'Precision',precision,'Recall',recall_1)
        i=i+1
        accuracy_v.append(accuracy)
    return accuracy_v,sum(accuracy_v)/10


##subject

***Here we showed with one subject for markov transitionfield, you can check other subjects by changing the input path (.csv file) and  replacing mark_img() function with gram_diff(),gram_summ() for FMRI_GramianAngularField.***

In [ ]:
coc_1=pd.read_csv('/content/Final_data_Split/Subject1/COCO.csv')
img_1=pd.read_csv('/content/Final_data_Split/Subject1/ImageNet.csv')
sun_1=pd.read_csv('/content/Final_data_Split/Subject1/SUN.csv')

In [ ]:
os.mkdir('/content/out1')

In [ ]:
batch=8

###**COCO_vs_sun**

In [ ]:
df_c=coc_1.iloc[:,:16]
df_c['label']=0 #COCO

df_s=sun_1.iloc[:,:16]
df_s['label']=1 #SUN
df_C_S=pd.concat([df_c,df_s])
df_C_S=res_ind(df_C_S)


In [ ]:
x,y=mark_img(df_C_S)
x.shape,y.shape

In [ ]:
model_1 = Model_1()
kf=10
floder_path='/content/out1/'
model_file='model_coco_vs_sun'
Kfloding(kf,model_1,x,y,floder_path,model_file,'subject_1',20,batch)

###**Imagenet_vs_sun**

In [ ]:
df_i=img_1.iloc[:,:16]
df_i['label']=0 #COCO

df_s=sun_1.iloc[:,:16]
df_s['label']=1 #SUN
df_I_S=pd.concat([df_i,df_s])
df_I_S=res_ind(df_I_S)


In [ ]:
x,y=mark_img(df_I_S)
x.shape,y.shape

In [ ]:
model_1 = Model_1()
kf=10
floder_path='/content/out1/'
model_file='model_image_vs_sun'
Kfloding(kf,model_1,x,y,floder_path,model_file,'subject_1',20,batch)

###**coco_vs_imagenet**


In [ ]:
df_i=img_1.iloc[:,:16]
df_i['label']=0 #img

df_c=coc_1.iloc[:,:16]
df_c['label']=1 #coco
df_I_C=pd.concat([df_c,df_i])
df_I_C=res_ind(df_I_C)


In [ ]:
x,y=mark_img(df_I_C)
x.shape,y.shape

In [ ]:
model_1 = Model_1()
kf=10
floder_path='/content/out1/'
model_file='model_coco_vs_imagenet'
Kfloding(kf,model_1,x,y,floder_path,model_file,'subject_1',50,batch)

###coco_vs_imagenet_vs_sun

In [ ]:
df_c=coc_1.iloc[:,:16]
df_c['label']=0
df_i=img_1.iloc[:,:16]
df_i['label']=1
df_s=sun_1.iloc[:,:16]
df_s['label']=2
df_total=pd.concat([df_i,df_c,df_s])
df_total=res_ind(df_total)
dat = df_total.copy()

In [ ]:
x,y=mark_img(dat)
x.shape,y.shape

In [ ]:
#model_1 = Model_1()
kf=10
floder_path='/content/out1/'
model_file='model_coco_vs_imagenet_sun'
kf_flod_three(kf,x,y,floder_path,model_file,'subject_1',50,batch)


#parallel cnn

##functions

In [ ]:
def interpolation(data):
  data1=data.iloc[:,:16]
  data1 = data1.replace(0,np.nan)
  data2=data1.T
  data2.reset_index(inplace=True)
  s1 = data2.interpolate(method='spline',order=1)
  s1.drop(['index'],axis=1,inplace=True)
  s2 = s1.T
  return s2

In [ ]:
#Reset index
def res_ind(df):
  df=df.reset_index()
  df=df.drop(['index'], axis=1)
  return df

In [ ]:
def Model_par_two():
    input_1 = Input(shape=(16,16,1))
    input_2 = Input(shape=(16,16,1))
    input_3 = Input(shape=(16,16,1))


    # First parallel convolutional network
    conv_1 = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(input_1)
    # Additional layers for the first parallel network
    max_p_1=MaxPooling2D((2, 2))(conv_1)
    conv_2 = Conv2D(filters=64, kernel_size=(2,2), activation='relu')(max_p_1)
    max_p_2=MaxPooling2D((2, 2))(conv_2)

    # Second parallel convolutional network
    conv_11 = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(input_2)
    # Additional layers for the second parallel network
    max_p_11=MaxPooling2D((2, 2))(conv_11)
    conv_22 = Conv2D(filters=64, kernel_size=(2,2), activation='relu')(max_p_11)
    max_p_22=MaxPooling2D((2, 2))(conv_22)

    # Second parallel convolutional network
    conv_3 = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(input_3)
    # Additional layers for the second parallel network
    max_p_31=MaxPooling2D((2, 2))(conv_3)
    conv_32 = Conv2D(filters=64, kernel_size=(2,2), activation='relu')(max_p_31)
    max_p_33=MaxPooling2D((2, 2))(conv_32)



    # Concatenate the output tensors from both parallel networks
    merged = Concatenate()([max_p_2, max_p_22,max_p_33])

    # Additional layers or operations on the merged output
    merged_1 = Flatten()(merged)
    merged_22 = Dense(128,activation='relu')(merged_1)
    merged_2 = Dense(32,activation='relu')(merged_22)

    merged_3 = Dense(8,activation='relu')(merged_2)
    merged_4 = Dense(1,activation='sigmoid')(merged_3)

    # Create the model
    model = tf.keras.models.Model(inputs=[input_1, input_2,input_3], outputs=merged_4)
    # Compile the model and specify loss function, optimizer, etc.
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy','Precision','Recall'])
    # Print the model summary
    #model.summary()
    return model

In [ ]:
def Kfloding(kf,model,x,y,x1,y1,x2,y2,floder_path,model_file,subject,epochs,batch_size):
  callback_2 = ModelCheckpoint(floder_path +str(subject)+'__'+str(model_file)+'.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
  callback_3 = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
  callback_1 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

  cv = StratifiedKFold(n_splits=int(kf),shuffle=True,random_state=98)
  #fig1 = plt.figure(figsize=[12,12])
  #ax1 = fig1.add_subplot(111,aspect = 'equal')
  accuracy_v=[]
  y_true=[]
  y_predict = []
  tprs = []
  aucs = []
  mean_fpr = np.linspace(0,1,100)
  i = 1
  for train_index,test_index in cv.split(x,y):
      x_train,x_test=x[train_index],x[test_index]
      y_train,y_test=y[train_index],y[test_index]

      x_train_1,x_test_1=x1[train_index],x1[test_index]
      y_train_1,y_test_1=y1[train_index],y1[test_index]

      x_train_2,x_test_2=x2[train_index],x2[test_index]
      y_train_2,y_test_2=y2[train_index],y2[test_index]

      model.fit([x_train, x_train_1,x_train_2], y_train,batch_size=batch_size,
                          epochs=epochs,
                          validation_data=([x_test,x_test_1,x_test_2],y_test),
                          callbacks=[callback_2])
      model = load_model(floder_path +str(subject)+'__'+str(model_file)+'.h5')
      print('Model evaluation ',model.evaluate([x_test,x_test_1,x_test_2],y_test))
      loss,accuracy,precision,recall_1=model.evaluate([x_test,x_test_1,x_test_2],y_test)
      print('kf_fold',i," ", 'loss',loss,'Accurary',accuracy,'Precision',precision,'Recall',recall_1)
      accuracy_v.append(accuracy)
  return accuracy_v,sum(accuracy_v)/10


'\n      res = model.evaluate(x_test,y_test)\n      #accuracy.append(res[1] * 100)\n      prediction = model.predict(x_test)\n      y_pred1 = (model.predict(x_test) > 0.5).astype(int)\n      #y_pred1 = np.array(prediction >= 0.5, dtype = int)\n      y_predict.append(y_pred1)\n      y_true.append(y_test)\n\n      fpr, tpr, t = roc_curve(y_test, prediction)\n      tprs.append(interp(mean_fpr, fpr, tpr))\n      roc_auc = auc(fpr, tpr)\n      aucs.append(roc_auc)\n      #plt.plot(fpr, tpr, lw=2, alpha=0.3, label=\'ROC fold %d (AUC = %0.2f)\' % (i, roc_auc))\n      i= i+1\n\n\n  mean_tpr = np.mean(tprs, axis=0)\n  mean_auc = auc(mean_fpr, mean_tpr)\n  plt.plot(mean_fpr, mean_tpr, color=\'blue\',\n          label=r\'Mean ROC (AUC = %0.2f )\' % (mean_auc),lw=2,)# alpha=1)\n\n  plt.plot([0,1],[0,1],\'k--\',lw = 2)\n  plt.xlim([0.0, 1.0])\n  plt.ylim([0.0, 1.05])\n  plt.xlabel(\'False Positive Rate\')\n  plt.ylabel(\'True Positive Rate\')\n  plt.title(\'ROC\')\n  plt.legend(loc="lower right")\n

In [ ]:
def Model_par_three():
    input_1 = Input(shape=(16,16,1))
    input_2 = Input(shape=(16,16,1))
    input_3 = Input(shape=(16,16,1))


    # First parallel convolutional network
    conv_1 = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(input_1)
    # Additional layers for the first parallel network
    max_p_1=MaxPooling2D((2, 2))(conv_1)
    conv_2 = Conv2D(filters=64, kernel_size=(2,2), activation='relu')(max_p_1)
    max_p_2=MaxPooling2D((2, 2))(conv_2)

    # Second parallel convolutional network
    conv_11 = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(input_2)
    # Additional layers for the second parallel network
    max_p_11=MaxPooling2D((2, 2))(conv_11)
    conv_22 = Conv2D(filters=64, kernel_size=(2,2), activation='relu')(max_p_11)
    max_p_22=MaxPooling2D((2, 2))(conv_22)

    # Second parallel convolutional network
    conv_3 = Conv2D(filters=32, kernel_size=(2,2), activation='relu')(input_3)
    # Additional layers for the second parallel network
    max_p_31=MaxPooling2D((2, 2))(conv_3)
    conv_32 = Conv2D(filters=64, kernel_size=(2,2), activation='relu')(max_p_31)
    max_p_33=MaxPooling2D((2, 2))(conv_32)



    # Concatenate the output tensors from both parallel networks
    merged = Concatenate()([max_p_2, max_p_22,max_p_33])

    # Additional layers or operations on the merged output
    merged_1 = Flatten()(merged)
    merged_22 = Dense(128,activation='relu')(merged_1)
    merged_2 = Dense(32,activation='relu')(merged_22)
    merged_3 = Dense(8,activation='relu')(merged_2)
    merged_4 = Dense(3,activation='softmax')(merged_3)

    # Create the model
    model = tf.keras.models.Model(inputs=[input_1, input_2,input_3], outputs=merged_4)
    # Compile the model and specify loss function, optimizer, etc.
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy','Precision','Recall'])
    # Print the model summary
    #model.summary()
    return model

In [ ]:
def mark_img(dat):
  mtf = MarkovTransitionField(image_size=16)
  MTF = mtf.fit_transform(dat.iloc[:,:-1])
  x = MTF.reshape(MTF.shape[0], 16, 16,1)
  y=dat['label']
  return x,y


In [ ]:
def gram_diff(dat):
  gram = GramianAngularField(image_size=16, method='difference')
  gram_t = gram.fit_transform(dat.iloc[:,:-1])
  x = gram_t.reshape(gram_t.shape[0], 16, 16,1)
  y=dat['label']
  return x,y

In [ ]:
def gram_summ(dat):
  gram = GramianAngularField(image_size=16, method='summation')
  gram_t = gram.fit_transform(dat.iloc[:,:-1])
  x = gram_t.reshape(gram_t.shape[0], 16, 16,1)
  y=dat['label']
  return x,y

In [ ]:
def kf_flod_three(kf,x,y,x1,y1,x2,y2,model1,floder_path,model_file,subject,epochs,batch_size):

    callback_2 = ModelCheckpoint(floder_path +str(subject)+'__'+str(model_file)+'.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    callback_3 = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
    callback_1 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

    cv = StratifiedKFold(n_splits=int(kf),shuffle=True,random_state=98)
    #fig1 = plt.figure(figsize=[12,12])
    #ax1 = fig1.add_subplot(111,aspect = 'equal')
    accuracy_v=[]
    y_true=[]
    y_predict = []
    pred_probs=[]
    tprs = []
    aucs = []
    fold_no=0
    #y_tf = tf.keras.utils.to_categorical(y, num_classes = 3)
    i=1
    for train_index,test_index in cv.split(x,y):
        x_train,x_test=x[train_index],x[test_index]
        y_train,y_test=y[train_index],y[test_index]

        x_train_1,x_test_1=x1[train_index],x1[test_index]
        y_train_1,y_test_1=y1[train_index],y1[test_index]

        x_train_2,x_test_2=x2[train_index],x2[test_index]
        y_train_2,y_test_2=y2[train_index],y2[test_index]

        y_train = tf.keras.utils.to_categorical(y_train, num_classes = 3)
        y_test = tf.keras.utils.to_categorical(y_test, num_classes = 3)
        model1.fit([x_train, x_train_1,x_train_2], y_train,batch_size=batch_size,
                            epochs=epochs,
                            validation_data=([x_test,x_test_1,x_test_2],y_test),
                            callbacks=[callback_2])
        model1 = load_model(floder_path +str(subject)+'__'+str(model_file)+'.h5')
        #scores = model.evaluate(x_test, y_test, verbose=0)
        #print(scores)
        #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        #   print("***************************************************\n")
        loss,accuracy,precision,recall_1=model1.evaluate([x_test,x_test_1,x_test_2],y_test)
        print('kf_fold',i," ", 'loss',loss,'Accurary',accuracy,'Precision',precision,'Recall',recall_1)
        i=i+1
        accuracy_v.append(accuracy)
    return accuracy_v,sum(accuracy_v)/10

'"\n        #accuracy.append(scores[1] * 100)\n        #loss_per_fold.append(scores[0])\n\n        y_pred = model1.predict(x_test)\n        pred_probs.extend(y_pred)\n        fin_lst = []\n        for i in y_pred:\n            fin_lst.append(np.argmax(i, axis=0))\n\n        fin_lst = np.array(fin_lst)\n        fin_lst = fin_lst.reshape((-1,1))\n    #     y_hat = np.array(y_pred >= 0.5, dtype = int)\n        #true_labels.extend(y[test])\n        y_true.extend(y_test)\n        y_predict.extend(fin_lst)\n\n\n        # Increase fold number\n        fold_no = fold_no + 1\n\n    ly_test = label_binarize(y_true, classes=[0, 1, 2])\n    test_yhat = label_binarize(y_predict, classes=[0, 1, 2])\n    n_classes = 3\n    fpr = dict()\n    tpr = dict()\n    roc_auc = dict()\n    for i in range(n_classes):\n        fpr[i], tpr[i], _ = roc_curve(ly_test[:, i], test_yhat[:, i])\n        roc_auc[i] = auc(fpr[i], tpr[i])\n\n    # Compute micro-average ROC curve and ROC area\n    fpr["micro"], tpr["micro"

##subject

In [ ]:
coc_1=pd.read_csv('/content/Final_data_Split/Subject1/COCO.csv')
img_1=pd.read_csv('/content/Final_data_Split/Subject1/ImageNet.csv')
sun_1=pd.read_csv('/content/Final_data_Split/Subject1/SUN.csv')


In [ ]:
sun_1 = interpolation(sun_1)

sun_1.columns = ['img1', 'img2', 'img3', 'img4', 'img5', 'img6', 'img7', 'img8', 'img9',
       'img10', 'img11', 'img12', 'img13', 'img14', 'img15', 'img16']
sun_1

In [ ]:
os.mkdir('/content/out1')
batch=8

###**COCO_vs_sun**

In [ ]:
df_c=coc_1.iloc[:,:16]
df_c['label']=0 #COCO

df_s=sun_1.iloc[:,:16]
df_s['label']=1 #SUN
df_C_S=pd.concat([df_c,df_s])
df_C_S=res_ind(df_C_S)


In [ ]:
x,y=mark_img(df_C_S)
x.shape,y.shape

In [ ]:
x1,y1=gram_diff(df_C_S)
x1.shape,y1.shape


In [ ]:
x2,y2=gram_summ(df_C_S)
x1.shape,y1.shape

In [ ]:
model_1 = Model_par_two()
kf=10
floder_path='/content/out1/'
model_file='model_coco_vs_sun'
Kfloding(kf,model_1,x,y,x1,y1,x2,y2,floder_path,model_file,'subject_1',20,batch)


###**Imagenet_vs_sun**

In [ ]:
df_i=img_1.iloc[:,:16]
df_i['label']=0 #COCO

df_s=sun_1.iloc[:,:16]
df_s['label']=1 #SUN
df_I_S=pd.concat([df_i,df_s])
df_I_S=res_ind(df_I_S)


In [ ]:
x,y=mark_img(df_I_S)
x.shape,y.shape

In [ ]:
x1,y1=gram_diff(df_I_S)
x1.shape,y1.shape


In [ ]:
x2,y2=gram_summ(df_I_S)
x2.shape,y2.shape


In [ ]:
kf=10
floder_path='/content/out1/'
model_file='model_image_vs_sun'
model_1 = Model_par_two()
Kfloding(kf,model_1,x,y,x1,y1,x2,y2,floder_path,model_file,'subject_1',20,batch)

###**coco_vs_imagenet**


In [ ]:
df_i=img_1.iloc[:,:16]
df_i['label']=0 #COCO

df_c=coc_1.iloc[:,:16]
df_c['label']=1 #SUN
df_I_C=pd.concat([df_i,df_c])
df_I_C=res_ind(df_I_C)


In [ ]:
x,y=mark_img(df_I_C)
x.shape,y.shape

In [ ]:
x1,y1=gram_diff(df_I_C)
x1.shape,y1.shape

In [ ]:
x2,y2=gram_summ(df_I_C)
x2.shape,y2.shape

In [ ]:
kf=10
floder_path='/content/out1/'
model_file='model_coco_vs_imagenet'
model_1 = Model_par_two()
Kfloding(kf,model_1,x,y,x1,y1,x2,y2,floder_path,model_file,'subject_1',50,batch)

###coco_vs_imagenet_vs_sun

In [ ]:
df_c=coc_1.iloc[:,:16]
df_c['label']=0
df_i=img_1.iloc[:,:16]
df_i['label']=1
df_s=sun_1.iloc[:,:16]
df_s['label']=2
df_total=pd.concat([df_i,df_c,df_s])
df_total=res_ind(df_total)
dat = df_total.copy()

In [ ]:
x,y=mark_img(dat)
x.shape,y.shape

In [ ]:
x1,y1=gram_diff(dat)
x1.shape,y1.shape

In [ ]:
x2,y2=gram_summ(dat)
x2.shape,y2.shape

In [ ]:
#model_1 = Model_1()
kf=10
floder_path='/content/out1/'
model_file='model_coco_vs_imagenet_sun'

model_1 = Model_par_three()
kf_flod_three(kf,x,y,x1,y1,x2,y2,model_1,floder_path,model_file,'subject_1',50,batch)